In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

In [3]:
books = pd.read_csv("books.csv",sep=";",on_bad_lines="skip", encoding='latin-1')
books.head()

/tmp/ipykernel_4665/1547265083.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("books.csv",sep=";",on_bad_lines="skip", encoding='latin-1')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
books=books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Image-URL-L']]
books.rename(columns={'ISBN':'isbn','Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher','Image-URL-L':'imageurl'},inplace=True)
books.head()

,isbn,title,author,year,publisher,imageurl
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [5]:
users=pd.read_csv("users.csv",sep=";",on_bad_lines="skip",encoding='latin-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
users.rename(columns={'User-ID':'userid','Location':'location','Age':'age'},inplace=True)
users.head()

,userid,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
ratings=pd.read_csv("ratings.csv",sep=";",on_bad_lines="skip",encoding='latin-1')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [8]:
ratings.rename(columns={'User-ID':'userid','ISBN':'isbn','Book-Rating':'rating'},inplace=True)
ratings.head()

,userid,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
print(books.shape,users.shape,ratings.shape,sep='\n')

(271360, 6)
(278858, 3)
(1149780, 3)


In [10]:
ratings['userid'].value_counts()

userid
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
119573        1
276706        1
276697        1
276679        1
276676        1
Name: count, Length: 105283, dtype: int64

In [11]:
x=ratings['userid'].value_counts() > 150
credible=x[x].index
ratings=ratings[ratings['userid'].isin(credible)]
ratings.shape

(582644, 3)

In [12]:
books_ratings=ratings.merge(books,on='isbn')
books_ratings.head()

,userid,isbn,rating,title,author,year,publisher,imageurl
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


In [13]:
number_ratings=books_ratings.groupby('title')['rating'].count().reset_index()
number_ratings.rename(columns={'rating':'num_ratings'},inplace=True)
number_ratings.head()

,title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,3
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [14]:
final_ratings=books_ratings.merge(number_ratings,on='title')
final_ratings.head()

,userid,isbn,rating,title,author,year,publisher,imageurl,num_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,88
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,15


In [15]:
final_ratings.drop_duplicates(['userid','title'],inplace=True)
final_ratings.dropna()

,userid,isbn,rating,title,author,year,publisher,imageurl,num_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,88
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,15
...,...,...,...,...,...,...,...,...,...
539222,276680,1884910033,0,Fiction Writer's Workshop,Josip Novakovich,1995,F &amp; W Pubns,http://images.amazon.com/images/P/1884910033.0...,5
539223,276680,1888173408,7,Fairy Tale (Common Reader Editions),Alice Thomas Ellis,2001,Akadine Press,http://images.amazon.com/images/P/1888173408.0...,1
539224,276680,1888173564,8,Parnassus on Wheels (Common Reader Editions),Christopher Morley,2000,Joiner/Oriel Inc,http://images.amazon.com/images/P/1888173564.0...,2
539225,276680,1888173572,0,Haunted Bookshop (Common Reader Editions),Christopher Morley,2000,Akadine Press,http://images.amazon.com/images/P/1888173572.0...,2


In [16]:
books_pivot=final_ratings.pivot_table(columns="userid",index="title",values="rating")
books_pivot

userid,254,1733,1903,2033,2110,2276,2766,2891,2977,3363,...,274808,275414,275970,276018,276680,277427,277478,277639,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Always Have Popsicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple Magic (The Collector's series),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beyond IBM: Leadership Marketing and Finance for the 1990s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Clifford Visita El Hospital (Clifford El Gran Perro Colorado),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ã?Â?lpiraten.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ã?Â?rger mit Produkt X. Roman.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
books_pivot.shape

(170111, 1223)

In [20]:
books_pivot.fillna(0,inplace=True)

In [22]:
books_sparse=csr_matrix(books_pivot)
type(books_sparse)

scipy.sparse._csr.csr_matrix

In [26]:
model = NearestNeighbors(algorithm='brute')
model.fit(books_sparse)

NearestNeighbors(algorithm='brute')

In [28]:
books_pivot.iloc[220,:]

userid
254       0.0
1733      0.0
1903      0.0
2033      0.0
2110      0.0
         ... 
277427    0.0
277478    0.0
277639    0.0
278188    0.0
278418    0.0
Name: 100 Treasures of Buckingham Palace, Length: 1223, dtype: float64

In [30]:
dist,suggest=model.kneighbors(books_pivot.iloc[220,:].values.reshape(1,-1),n_neighbors=6)
dist

array([[0., 0., 0., 0., 0., 0.]])

In [32]:
for i in range(len(suggest)):
    print(books_pivot.index[suggest[i]])

Index(['100 Treasures of Buckingham Palace',
       'Alice in Wonderland (Puffin Classics)',
       'Antiques Roadshow Primer : The Introductory Guide to Antiques and Collectibles from the Most-Watched Series on PBS',
       'Frommer's Comprehensive Travel Guide Washington, D.C '95 (Frommer's City Guides)',
       'Frommer's 2000 London from $85 a Day: The Ultimate Guide to Comfortable Low-Cost Travel (Frommer's London from $.. a Day, 2000)',
       'A History of Private Life: Revelations of the Medieval World (History of Private Life (Paperback))'],
      dtype='object', name='title')


In [33]:
book_names=books_pivot.index
book_names

Index([' A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)',
       ' Always Have Popsicles', ' Apple Magic (The Collector's series)',
       ' Beyond IBM: Leadership Marketing and Finance for the 1990s',
       ' Clifford Visita El Hospital (Clifford El Gran Perro Colorado)',
       ' Dark Justice', ' Deceived',
       ' Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth',
       ' Final Fantasy Anthology: Official Strategy Guide (Brady Games)',
       ' Flight of Fancy: American Heiresses (Zebra Ballad Romance)',
       ...
       'Ã?ber den Wunsch, sich wohlzufÃ¼hlen: Geschichten',
       'Ã?coute ma diffÃ©rence (Le Temps des femmes)',
       'Ã?ngeles fugaces (Falling Angels)', 'Ã?Â?. Kolumnen.',
       'Ã?Â?ber das Fernsehen.',
       'Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.', 'Ã?Â?lpiraten.',
       'Ã?Â?rger mit Produkt X. Roman.', 'Ã?Â?stlich der Berge.',
       'Ã?Â

In [37]:
book_name=[]
for book_id in suggest:
    book_name.append(books_pivot.index[book_id])

ids_index=[]
for name in book_name[0]:
    ids=np.where(final_ratings['title']==name)[0][0]
    ids_index.append(ids)

for idx in ids_index:
    url=final_ratings.iloc[idx]['imageurl']
    print(url)

http://images.amazon.com/images/P/1902163923.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0140350381.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0761116249.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0671883836.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0028634764.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0674400011.01.LZZZZZZZ.jpg


In [40]:
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_ratings,open('artifacts/final_ratings.pkl','wb'))
pickle.dump(books_pivot,open('artifacts/books_pivot.pkl','wb'))

In [52]:
def recommend(book_name):
    book_id = np.where(books_pivot.index==book_name)[0][0]
    dist, suggest = model.kneighbors(books_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)

    for i in range(len(suggest)):
        books= books_pivot.index[suggest[i]]
        for j in books:
            if j==book_name:
                print(f"You searched for -: {book_name}\n")
                print("The suggested books are -: \n")
            else:
                print(j)

In [53]:
recommend(' Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth')

You searched for -:  Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth

The suggested books are -: 

What to Expect When You're Expecting
A Christmas Carol (Apple Classics)
What Makes Popcorn Pop?: And Other Questions About the World Around Us
The Christmas Alphabet
Peace Pilgrim: Her Life and Work in Her Own Words
